Library:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Load Data & Data Processing

In [3]:
train_data_df = pd.read_csv('train.csv')
train_data_df.dropna(inplace=True)

# Note that data is DataFrame type
print(train_data_df.shape)

feature_list = train_data_df.columns.tolist()
print(feature_list)

# change category into numeric types
category_labels = {"jam":0, "compo":1}
train_data_df.category = [category_labels[item] for item in train_data_df.category]

# remove incorrect data
train_data_df = train_data_df.drop(train_data_df[train_data_df.label < 0].index)
train_data_df = train_data_df.drop(train_data_df[train_data_df.label > 5].index)

# remove irrelevant data
irrelevant_feature = ["id", "name", "slug", "path", "competition-num", "description", "published", "modified", "version", "links", "link-tags"]
train_data_df = train_data_df.drop(irrelevant_feature, axis=1)

train_x_df = train_data_df.iloc[:,:-1]
train_y_df = train_data_df.iloc[:,-1]

# centralize x
train_x_df = (train_x_df - train_x_df.min())/(train_x_df.max()-train_x_df.min())

# get dimension for input and output for training purpose
input_dim = train_x_df.shape[1]
output_dim = 6 # 0 to 5 stars
print('input dimension: ', input_dim)

train_x_tensor = torch.Tensor(train_x_df.to_numpy())
print('train_x format: ', train_x_tensor.shape, train_x_tensor.dtype)
train_y_tensor = torch.Tensor(train_y_df.to_numpy())
train_y_tensor = train_y_tensor.type(torch.int64)
print('train_y format: ', train_y_tensor.shape, train_y_tensor.dtype)
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)

# Split to Train and Validate sets using random_split
train_batch_size = 1
number_rows = len(train_x_tensor)    # The size of our dataset or the number of rows in excel table.
validate_split = int(number_rows * 0.2)
train_split = number_rows - validate_split
train_set, validate_set = random_split(train_dataset, [train_split, validate_split])

# Create Dataloader to read the data within batch sizes and put into memory.
train_dataloader = DataLoader(train_set, batch_size = train_batch_size, shuffle = True)
validate_dataloader = DataLoader(validate_set, batch_size = 1)
entire_dataloader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True)

(18263, 33)
['id', 'name', 'slug', 'path', 'competition-num', 'category', 'description', 'num-comments', 'published', 'modified', 'version', 'feedback-karma', 'ratings-given', 'ratings-received', 'links', 'link-tags', 'num-authors', 'prev-games', 'fun-average', 'innovation-average', 'theme-average', 'graphics-average', 'audio-average', 'humor-average', 'mood-average', 'fun-rank', 'innovation-rank', 'theme-rank', 'graphics-rank', 'audio-rank', 'humor-rank', 'mood-rank', 'label']
input dimension:  21
train_x format:  torch.Size([18263, 21]) torch.float32
train_y format:  torch.Size([18263]) torch.int64


In [ ]:
test_data_df = pd.read_csv('test.csv')
labels = [0] * test_data_df.shape[0]
test_data_df["label"] = labels
print(test_data_df.shape)

# get ids
ids = test_data_df["id"].tolist()

# change category into numeric types
category_labels = {"jam":0, "compo":1}
test_data_df.category = [category_labels[item] for item in test_data_df.category]

# remove irrelevant data
irrelevant_feature = ["id", "name", "slug", "path", "competition-num", "description", "published", "modified", "version", "links", "link-tags"]
test_x_df = test_data_df.drop(irrelevant_feature, axis=1)

test_x_df = test_x_df.iloc[:,:-1]
test_y_df = test_x_df.iloc[:,-1]

# centralize x
test_x_df = (test_x_df - test_x_df.min())/(test_x_df.max()-test_x_df.min())

test_x_tensor = torch.Tensor(test_x_df.to_numpy())
print('train_x format: ', test_x_tensor.shape, test_x_tensor.dtype)
test_y_tensor = torch.Tensor(test_y_df.to_numpy())
test_y_tensor = test_y_tensor.type(torch.int64)
print('train_y format: ', test_y_tensor.shape, test_y_tensor.dtype)
test_dataset = TensorDataset(test_x_tensor, test_y_tensor)

# Create Dataloader to read the data within batch sizes and put into memory.
test_dataloader = DataLoader(test_dataset, batch_size = 1)

(4959, 33)
train_x format:  torch.Size([4959, 21]) torch.float32
train_y format:  torch.Size([4959]) torch.int64


In [ ]:
def train_one_epoch(epoch_index):
  train_total_correct = 0

  for i, data in enumerate(train_dataloader):
      # Every data instance is an input + label pair
      train_inputs, train_labels = data
      train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)

      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      train_outputs = model(train_inputs)

      # Computer Correctness
      train_predict = train_outputs.argmax(dim=1)
      train_total_correct += torch.eq(train_predict, train_labels).sum().item()

      # Compute the loss and its gradients
      train_loss = loss_fn(train_outputs, train_labels)
      train_loss.backward()

      # Adjust learning weights
      optimizer.step()

      # if (i % 1000 == 999):
      #   print("first {} accuracy: {}".format(i + 1, train_total_correct/(train_batch_size * (i + 1))))

  return train_total_correct/(train_batch_size * (i + 1))

In [ ]:
def train_multi_epoch(EPOCHS = 5):
  epoch_number = 0

  train_accuracys = []
  validate_accuracys = []

  for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    train_accuracy = train_one_epoch(epoch_number)
    train_accuracys.append(train_accuracy)

    # We don't need gradients on to do reporting
    model.train(False)

    validate_total_correct = 0
    for data in validate_dataloader:
        validate_inputs, validate_labels = data
        validate_inputs, validate_labels = validate_inputs.to(device), validate_labels.to(device)
        validate_outputs = model(validate_inputs)

        # test correctness
        validate_predict = validate_outputs.argmax(dim=1)
        validate_total_correct += torch.eq(validate_predict, validate_labels).sum().item()

    validate_accuracy = validate_total_correct/len(validate_dataloader)
    validate_accuracys.append(validate_accuracy)

    print("Train Accuracy: {}".format(train_accuracy))
    print("Validate Accuracy: {}".format(validate_accuracy))

    epoch_number += 1
  return validate_accuracys

In [ ]:
class NN_1(nn.Module):
  def __init__(self):
    super(NN_1, self).__init__()
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(input_dim, 64)
    self.linear1_dp = nn.Dropout(p=0.5)

    self.linear2 = nn.Linear(64, 32)
    self.linear2_dp = nn.Dropout(p=0.5)

    self.linear3 = nn.Linear(32, 16)
    self.linear3_dp = nn.Dropout(p=0.5)

    self.linearfinal = nn.Linear(16, output_dim)

  def forward(self, x):
    x = self.flatten(x)
    # x = self.linear1_dp(F.relu(self.linear1(x)))
    # x = self.linear2_dp(F.relu(self.linear2(x)))
    # x = self.linear3_dp(F.relu(self.linear3(x)))
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = self.linearfinal(x)
    return x

model = NN_1()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch()

EPOCH 1:
Train Accuracy: 0.8839915132434467
Test Accuracy: 0.920591456736035
EPOCH 2:
Train Accuracy: 0.9024022996372596
Test Accuracy: 0.9208652792990142
EPOCH 3:
Train Accuracy: 0.9073300937649716
Test Accuracy: 0.8951259583789705
EPOCH 4:
Train Accuracy: 0.9051399630415441
Test Accuracy: 0.8674698795180723
EPOCH 5:
Train Accuracy: 0.9110943809458627
Test Accuracy: 0.8992332968236583


[0.920591456736035,
 0.9208652792990142,
 0.8951259583789705,
 0.8674698795180723,
 0.8992332968236583]

In [ ]:
class NN_2(nn.Module):
  def __init__(self):
    super(NN_2, self).__init__()
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(input_dim, 32)
    self.linear1_dp = nn.Dropout(p=0.5)

    self.linear2 = nn.Linear(32, 32)
    self.linear2_dp = nn.Dropout(p=0.5)

    self.linear3 = nn.Linear(32, 16)
    self.linear3_dp = nn.Dropout(p=0.5)

    self.linearfinal = nn.Linear(16, output_dim)

  def forward(self, x):
    x = self.flatten(x)
    # x = self.linear1_dp(F.relu(self.linear1(x)))
    # x = self.linear2_dp(F.relu(self.linear2(x)))
    # x = self.linear3_dp(F.relu(self.linear3(x)))
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = self.linearfinal(x)
    return x

model = NN_2()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch()

EPOCH 1:
Train Accuracy: 0.8842652795838751
Test Accuracy: 0.9225082146768894
EPOCH 2:
Train Accuracy: 0.9034973649989734
Test Accuracy: 0.9181270536692223
EPOCH 3:
Train Accuracy: 0.910067757169256
Test Accuracy: 0.9211391018619934
EPOCH 4:
Train Accuracy: 0.9090411333926494
Test Accuracy: 0.8888280394304491
EPOCH 5:
Train Accuracy: 0.9121210047224694
Test Accuracy: 0.9167579408543264


[0.9225082146768894,
 0.9181270536692223,
 0.9211391018619934,
 0.8888280394304491,
 0.9167579408543264]

In [ ]:
class NN_3(nn.Module):
  def __init__(self):
    super(NN_3, self).__init__()
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(input_dim, 32)
    self.linear1_dp = nn.Dropout(p=0.5)

    self.linear2 = nn.Linear(32, 16)
    self.linear2_dp = nn.Dropout(p=0.5)

    self.linearfinal = nn.Linear(16, output_dim)

  def forward(self, x):
    x = self.flatten(x)
    # x = self.linear1_dp(F.relu(self.linear1(x)))
    # x = self.linear2_dp(F.relu(self.linear2(x)))
    x = F.relu(self.linear1(x))
    x = F.tanh(self.linear2(x))
    x = self.linearfinal(x)
    return x

In [ ]:
model = NN_3()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch(10)

EPOCH 1:
Train Accuracy: 0.887824242009445
Test Accuracy: 0.9194961664841182
EPOCH 2:
Train Accuracy: 0.8959687906371911
Test Accuracy: 0.9000547645125958
EPOCH 3:
Train Accuracy: 0.9004859352542605
Test Accuracy: 0.8995071193866374
EPOCH 4:
Train Accuracy: 0.8978851550201903
Test Accuracy: 0.914841182913472
EPOCH 5:
Train Accuracy: 0.8930258024775853
Test Accuracy: 0.904709748083242
EPOCH 6:
Train Accuracy: 0.8951474916159058
Test Accuracy: 0.9208652792990142
EPOCH 7:
Train Accuracy: 0.89836424611594
Test Accuracy: 0.901697699890471
EPOCH 8:
Train Accuracy: 0.90144411744576
Test Accuracy: 0.9096385542168675
EPOCH 9:
Train Accuracy: 0.8991171035521183
Test Accuracy: 0.9044359255202629
EPOCH 10:
Train Accuracy: 0.8956265827116556
Test Accuracy: 0.8880065717415115


[0.9194961664841182,
 0.9000547645125958,
 0.8995071193866374,
 0.914841182913472,
 0.904709748083242,
 0.9208652792990142,
 0.901697699890471,
 0.9096385542168675,
 0.9044359255202629,
 0.8880065717415115]

In [ ]:
model = NN_3()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_multi_epoch(10)

EPOCH 1:
Train Accuracy: 0.8608582574772432
Test Accuracy: 0.8409090909090909
EPOCH 2:
Train Accuracy: 0.9095202244883992
Test Accuracy: 0.916210295728368
EPOCH 3:
Train Accuracy: 0.9070563274245431
Test Accuracy: 0.9200438116100766
EPOCH 4:
Train Accuracy: 0.9128738621586476
Test Accuracy: 0.9164841182913472
EPOCH 5:
Train Accuracy: 0.913147628499076
Test Accuracy: 0.9252464403066812
EPOCH 6:
Train Accuracy: 0.9177332147012525
Test Accuracy: 0.9216867469879518
EPOCH 7:
Train Accuracy: 0.9180069810416809
Test Accuracy: 0.9066265060240963
EPOCH 8:
Train Accuracy: 0.9183491889672165
Test Accuracy: 0.901697699890471
EPOCH 9:
Train Accuracy: 0.9188282800629662
Test Accuracy: 0.9337349397590361
EPOCH 10:
Train Accuracy: 0.9208815276161796
Test Accuracy: 0.932092004381161


[0.8409090909090909,
 0.916210295728368,
 0.9200438116100766,
 0.9164841182913472,
 0.9252464403066812,
 0.9216867469879518,
 0.9066265060240963,
 0.901697699890471,
 0.9337349397590361,
 0.932092004381161]

In [ ]:
class NN_4(nn.Module):
  def __init__(self):
    super(NN_4, self).__init__()
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(input_dim, 32)
    self.linear1_dp = nn.Dropout(p=0.5)

    self.linear2 = nn.Linear(32, 32)
    self.linear2_dp = nn.Dropout(p=0.5)

    self.linearfinal = nn.Linear(32, output_dim)

  def forward(self, x):
    x = self.flatten(x)
    # x = self.linear1_dp(F.relu(self.linear1(x)))
    # x = self.linear2_dp(F.relu(self.linear2(x)))
    x = F.relu(self.linear1(x))
    x = F.tanh(self.linear2(x))
    x = self.linearfinal(x)
    return x

In [ ]:
model = NN_4()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch(10)

EPOCH 1:


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Train Accuracy: 0.8805694339880912
Validate Accuracy: 0.8975903614457831
EPOCH 2:
Train Accuracy: 0.89836424611594
Validate Accuracy: 0.8986856516976999
EPOCH 3:
Train Accuracy: 0.8981589213606187
Validate Accuracy: 0.920591456736035
EPOCH 4:
Train Accuracy: 0.8937786599137636
Validate Accuracy: 0.8803395399780942
EPOCH 5:
Train Accuracy: 0.897132297584012
Validate Accuracy: 0.9236035049288062
EPOCH 6:
Train Accuracy: 0.8962425569776196
Validate Accuracy: 0.8491237677984665
EPOCH 7:
Train Accuracy: 0.8967216480733694
Validate Accuracy: 0.906078860898138
EPOCH 8:
Train Accuracy: 0.8963794401478338
Validate Accuracy: 0.8953997809419496
EPOCH 9:
Train Accuracy: 0.8985695708712613
Validate Accuracy: 0.8997809419496167
EPOCH 10:
Train Accuracy: 0.8961056738074054
Validate Accuracy: 0.9112814895947426


[0.8975903614457831,
 0.8986856516976999,
 0.920591456736035,
 0.8803395399780942,
 0.9236035049288062,
 0.8491237677984665,
 0.906078860898138,
 0.8953997809419496,
 0.8997809419496167,
 0.9112814895947426]

In [ ]:
model = NN_4()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_multi_epoch(10)

EPOCH 1:
Train Accuracy: 0.8729039764560947
Validate Accuracy: 0.9164841182913472
EPOCH 2:
Train Accuracy: 0.9093148997330778
Validate Accuracy: 0.9181270536692223
EPOCH 3:
Train Accuracy: 0.9127369789884334
Validate Accuracy: 0.9203176341730559
EPOCH 4:
Train Accuracy: 0.9178700978714667
Validate Accuracy: 0.8989594742606791
EPOCH 5:
Train Accuracy: 0.9181438642118952
Validate Accuracy: 0.9208652792990142
EPOCH 6:
Train Accuracy: 0.9203339949353228
Validate Accuracy: 0.9241511500547646
EPOCH 7:
Train Accuracy: 0.9189651632331804
Validate Accuracy: 0.9227820372398685
EPOCH 8:
Train Accuracy: 0.9219765929778934
Validate Accuracy: 0.9299014238773274
EPOCH 9:
Train Accuracy: 0.9223188009034289
Validate Accuracy: 0.9318181818181818
EPOCH 10:
Train Accuracy: 0.9208815276161796
Validate Accuracy: 0.9071741511500547


[0.9164841182913472,
 0.9181270536692223,
 0.9203176341730559,
 0.8989594742606791,
 0.9208652792990142,
 0.9241511500547646,
 0.9227820372398685,
 0.9299014238773274,
 0.9318181818181818,
 0.9071741511500547]

In [ ]:
model = NN_4()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch(10)

EPOCH 1:
Train Accuracy: 0.8937786599137636
Test Accuracy: 0.874315443592552
EPOCH 2:
Train Accuracy: 0.9108890561905414
Test Accuracy: 0.9225082146768894
EPOCH 3:
Train Accuracy: 0.9130107453288618
Test Accuracy: 0.9222343921139102
EPOCH 4:
Train Accuracy: 0.9138320443501472
Test Accuracy: 0.9244249726177437
EPOCH 5:
Train Accuracy: 0.9164328245842174
Test Accuracy: 0.9079956188389924
EPOCH 6:
Train Accuracy: 0.9173910067757169
Test Accuracy: 0.9257940854326396
EPOCH 7:
Train Accuracy: 0.9176647731161454
Test Accuracy: 0.9203176341730559
EPOCH 8:
Train Accuracy: 0.9162274998288961
Test Accuracy: 0.9227820372398685
EPOCH 9:
Train Accuracy: 0.9178700978714667
Test Accuracy: 0.9186746987951807
EPOCH 10:
Train Accuracy: 0.918075422626788
Test Accuracy: 0.9227820372398685


[0.874315443592552,
 0.9225082146768894,
 0.9222343921139102,
 0.9244249726177437,
 0.9079956188389924,
 0.9257940854326396,
 0.9203176341730559,
 0.9227820372398685,
 0.9186746987951807,
 0.9227820372398685]

Fit model with entrie dataset and predict

In [ ]:
def final_train_one_epoch(epoch_index):
  train_total_correct = 0

  for i, data in enumerate(entire_dataloader):
      # Every data instance is an input + label pair
      train_inputs, train_labels = data
      train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)

      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      train_outputs = model(train_inputs)

      # Computer Correctness
      train_predict = train_outputs.argmax(dim=1)
      train_total_correct += torch.eq(train_predict, train_labels).sum().item()

      # Compute the loss and its gradients
      train_loss = loss_fn(train_outputs, train_labels)
      train_loss.backward()

      # Adjust learning weights
      optimizer.step()

      # if (i % 1000 == 999):
      #   print("first {} accuracy: {}".format(i + 1, train_total_correct/(train_batch_size * (i + 1))))

  return train_total_correct/(train_batch_size * (i + 1))

def final_train_multi_epoch(EPOCHS = 5):
  epoch_number = 0

  entire_accuracys = []

  for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    train_accuracy = final_train_one_epoch(epoch_number)
    entire_accuracys.append(train_accuracy)

    print("Train Accuracy: {}".format(train_accuracy))

    epoch_number += 1
  return entire_accuracys

def predict():
  model.train(False)
  train_total_correct = 0
  predicts = []
  for data in test_dataloader:
    test_inputs,_ = data
    test_inputs = test_inputs.to(device)
    test_outputs = model(test_inputs)

    # test correctness
    test_predict = test_outputs.argmax(dim=1)
    predicts.append(test_predict.item())

  return predicts

In [ ]:
model = NN_4()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_multi_epoch(20)

predicts = predict()
print(predicts)
elements = {"id": ids, "label": predicts}
predict_df = pd.DataFrame(elements)
predict_df.to_csv('submission1.csv',index=False)

EPOCH 1:
Train Accuracy: 0.8757100814454862
Validate Accuracy: 0.9140197152245345
EPOCH 2:
Train Accuracy: 0.9090411333926494
Validate Accuracy: 0.9323658269441402
EPOCH 3:
Train Accuracy: 0.9132845116692903
Validate Accuracy: 0.9323658269441402
EPOCH 4:
Train Accuracy: 0.911847238382041
Validate Accuracy: 0.9153888280394304
EPOCH 5:
Train Accuracy: 0.9154062008076107
Validate Accuracy: 0.9222343921139102
EPOCH 6:
Train Accuracy: 0.9186229553076449
Validate Accuracy: 0.9107338444687842
EPOCH 7:
Train Accuracy: 0.9206762028608583
Validate Accuracy: 0.9323658269441402
EPOCH 8:
Train Accuracy: 0.9185545137225378
Validate Accuracy: 0.9093647316538883
EPOCH 9:
Train Accuracy: 0.9199917870097871
Validate Accuracy: 0.9194961664841182
EPOCH 10:
Train Accuracy: 0.9189651632331804
Validate Accuracy: 0.9112814895947426
EPOCH 11:
Train Accuracy: 0.9221134761481076
Validate Accuracy: 0.9370208105147865
EPOCH 12:
Train Accuracy: 0.9188282800629662
Validate Accuracy: 0.9359255202628697
EPOCH 13:
Trai

In [ ]:
model = NN_4()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_multi_epoch(20)

predicts = predict()
print(predicts)
elements = {"id": ids, "label": predicts}
predict_df = pd.DataFrame(elements)
predict_df.to_csv('submission2.csv',index=False)

EPOCH 1:
Train Accuracy: 0.881664499349805
Validate Accuracy: 0.926341730558598
EPOCH 2:
Train Accuracy: 0.902676065977688
Validate Accuracy: 0.9197699890470975
EPOCH 3:
Train Accuracy: 0.9043871056053658
Validate Accuracy: 0.9192223439211391
EPOCH 4:
Train Accuracy: 0.905755937307508
Validate Accuracy: 0.9230558598028478
EPOCH 5:
Train Accuracy: 0.906303469988365
Validate Accuracy: 0.9241511500547646
EPOCH 6:
Train Accuracy: 0.9106837314352201
Validate Accuracy: 0.9323658269441402
EPOCH 7:
Train Accuracy: 0.906987885839436
Validate Accuracy: 0.9271631982475356
EPOCH 8:
Train Accuracy: 0.9065772363287934
Validate Accuracy: 0.9252464403066812
EPOCH 9:
Train Accuracy: 0.9036342481691876
Validate Accuracy: 0.9312705366922235
EPOCH 10:
Train Accuracy: 0.9091095749777565
Validate Accuracy: 0.9077217962760131
EPOCH 11:
Train Accuracy: 0.9056874957224009
Validate Accuracy: 0.9266155531215772
EPOCH 12:
Train Accuracy: 0.9108206146054343
Validate Accuracy: 0.9123767798466593
EPOCH 13:
Train Acc

In [ ]:
model = NN_3()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_multi_epoch(20)

predicts = predict()
print(predicts)
elements = {"id": ids, "label": predicts}
predict_df = pd.DataFrame(elements)
predict_df.to_csv('submission3.csv',index=False)

EPOCH 1:
Train Accuracy: 0.8710560536582027
Validate Accuracy: 0.9266155531215772
EPOCH 2:
Train Accuracy: 0.9078091848607214
Validate Accuracy: 0.8647316538882804
EPOCH 3:
Train Accuracy: 0.9104099650947916
Validate Accuracy: 0.9301752464403067
EPOCH 4:
Train Accuracy: 0.9111628225309698
Validate Accuracy: 0.9274370208105148
EPOCH 5:
Train Accuracy: 0.911847238382041
Validate Accuracy: 0.9186746987951807
EPOCH 6:
Train Accuracy: 0.9148586681267538
Validate Accuracy: 0.9222343921139102
EPOCH 7:
Train Accuracy: 0.9171856820203956
Validate Accuracy: 0.9351040525739321
EPOCH 8:
Train Accuracy: 0.9154746423927178
Validate Accuracy: 0.9323658269441402
EPOCH 9:
Train Accuracy: 0.9182123057970023
Validate Accuracy: 0.9356516976998904
EPOCH 10:
Train Accuracy: 0.9203339949353228
Validate Accuracy: 0.9233296823658269
EPOCH 11:
Train Accuracy: 0.9178016562863596
Validate Accuracy: 0.9309967141292442
EPOCH 12:
Train Accuracy: 0.9210868523715009
Validate Accuracy: 0.9394852135815991
EPOCH 13:
Trai